# import

In [2]:
import torch
import torch.nn as nn
from vilt.modules import heads, objectives
import vilt.modules.vision_transformer as vit
import torch.nn.functional as F
import random
from typing import OrderedDict
import os
import pandas as pd
import numpy as np
from vilt.transforms import pixelbert_transform
from PIL import Image
from torchvision import transforms
from tqdm import tqdm
from torch.utils.data import DataLoader
import gc
import torch.optim as optim
from torch.optim import lr_scheduler
from collections import defaultdict
import wandb

from sklearn.model_selection import StratifiedKFold, KFold, StratifiedGroupKFold


/home/junsheng/.conda/envs/pytorch_junsheng_39/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# config

In [3]:


class config:
    debug = False
    exp_name = "vilt"
    seed = 101
    batch_size = 4096  # this is a desired batch size; pl trainer will accumulate gradients when per step batch is smaller.
    train_batch_size = 32
    valid_batch_size = 4
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    # root_path = r'E:\\Download\\xiangguan' # 存放数据的根目录
    root_path = r'/home/junsheng/data/xiangguan' # 存放数据的根目录
    n_fold = 5
    

    # Image setting
    train_transform_keys = ["pixelbert"]
    val_transform_keys = ["pixelbert"]
    img_size = 384
    max_image_len = -1
    patch_size = 32
    draw_false_image = 1
    image_only = False

    # Sensor
    senser_input_num = 11
    
    # Text Setting
    vqav2_label_size = 3129
    max_text_len = 40
    tokenizer = "bert-base-uncased"
    vocab_size = 30522 # vocabulary词汇数量
    whole_word_masking = False
    mlm_prob = 0.15
    draw_false_text = 0

    # Transformer Setting
    vit = "vit_base_patch32_384"
    hidden_size = 768  # 嵌入向量大小
    num_heads = 12
    num_layers = 12
    mlp_ratio = 4
    drop_rate = 0.1

    # Optimizer Setting
    optim_type = "adamw"
    learning_rate = 1e-3
    weight_decay = 1e-4 # 0.01 ->1e-4
    decay_power = 1
    max_epoch = 50
    max_steps = 25000
    warmup_steps = 2500
    end_lr = 0
    lr_mult = 1  # multiply lr for downstream heads
    T_max = 8000/train_batch_size*max_epoch

    # Downstream Setting
    get_recall_metric = False


    # below params varies with the environment
    data_root = ""
    log_dir = "result"
    per_gpu_batchsize = 0  # you should define this manually with per_gpu_batch_size=#
    num_gpus = 1
    num_nodes = 1
    load_path = "weights/vilt_200k_mlm_itm.ckpt"
    # load_path = "save_model_dict.pt"
    num_workers = 1
    precision = 16

# config = vars(config)
# config = dict(config)
config

if config.debug:
    config.max_epoch = 5

In [4]:
def setup_seed(seed):

    torch.manual_seed(seed)  # 为CPU设置随机种子
    np.random.seed(seed)  # Numpy module.
    random.seed(seed)  # Python random module.
    # torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
    torch.cuda.manual_seed(seed)  # 为当前GPU设置随机种子
    torch.cuda.manual_seed_all(seed)  # 为所有GPU设置随机种子
    #os.environ['PYTHONHASHSEED'] = str(seed)
    
setup_seed(config.seed)

# wandb

In [5]:
os.environ["WANDB_MODE"] = 'dryrun' # 离线模式
try:
    # wandb.log(key="*******") # if debug
    wandb.login() # storage in ~/.netrc file
    anonymous = None
except:
    anonymous = "must"
    print('\nGet your W&B access token from here: https://wandb.ai/authorize\n')


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


# 数据

In [8]:
df_xiangguan = pd.read_csv("xiangguan-corn-2.csv")
df_xiangguan['image_path'] = df_xiangguan['url'].map(lambda x:os.path.join(config.root_path,'pic',x.split('/')[-1]))
df_xiangguan['raw_label'] = df_xiangguan['label'].astype(dtype='int64')
# del df_xiangguan['label_name']
df_xiangguan.head()

,device_id,collect_time,url,picture_id,temperature,humidity,illuminance,soil_temperature,soil_humidity,pressure,wind_speed,photosynthetic,sun_exposure_time,COz,soil_ph,weather_id,label_name,label,image_path,raw_label
0,xiangguanD3,14/5/2021 04,https://daqing-haikang-image.oss-cn-hangzhou.a...,34.0,13.66,51.78,76.73,8.42,8.02,981.09,2.48,7.92,0.1,659.34,6.80,45288.0,出苗,0,/home/junsheng/data/xiangguan/pic/xiangguanD3-...,0
1,xiangguanD3,14/5/2021 04,https://daqing-haikang-image.oss-cn-hangzhou.a...,34.0,13.76,56.13,226.02,8.32,8.02,980.99,3.37,28.71,0.1,660.33,6.80,45292.0,出苗,0,/home/junsheng/data/xiangguan/pic/xiangguanD3-...,0
2,xiangguanD4,14/5/2021 04,https://daqing-haikang-image.oss-cn-hangzhou.a...,35.0,13.70,52.00,240.80,5.50,26.10,75.00,1.85,6.00,0.1,158.00,6.82,45289.0,出苗,0,/home/junsheng/data/xiangguan/pic/xiangguanD4-...,0
3,xiangguanD4,14/5/2021 04,https://daqing-haikang-image.oss-cn-hangzhou.a...,35.0,13.90,55.80,670.80,5.50,26.00,15.30,2.37,18.00,0.1,157.00,6.82,45293.0,出苗,0,/home/junsheng/data/xiangguan/pic/xiangguanD4-...,0
4,xiangguanD3,14/5/2021 05,https://daqing-haikang-image.oss-cn-hangzhou.a...,38.0,13.76,59.90,428.97,8.22,8.02,980.99,2.13,57.42,0.1,659.34,6.80,45296.0,出苗,0,/home/junsheng/data/xiangguan/pic/xiangguanD3-...,0


数据检查

In [9]:
# 检查图片下载的全不全
pic = df_xiangguan.image_path.map(lambda x:x.split('/')[-1]).unique()
print(len(pic))
file_ls = os.listdir("/home/junsheng/data/xiangguan/pic")
print(len(file_ls))
ret = list(set(pic) ^ set(file_ls))
print(ret) #差集
assert len(pic)==len(file_ls),"请检查下载的图片，缺了{}个".format(len(pic)-len(file_ls))


7997
7997
[]


归一化非object列

In [10]:
number_title = []
recorder = {}
for title in df_xiangguan:
    # print(df_xiangguan[title].head())
    if title == 'raw_label':
        continue
    if df_xiangguan[title].dtype != "object":
        
        number_title.append(title)
        x_min = df_xiangguan[title].min()
        x_max = df_xiangguan[title].max()
        # print(x_min,x_max)
        recorder[title] = (x_min,x_max)
        df_xiangguan[title] = df_xiangguan[title].map(lambda x:(x-x_min)/(x_max - x_min))
print(number_title)
print(recorder)

['picture_id', 'temperature', 'humidity', 'illuminance', 'soil_temperature', 'soil_humidity', 'pressure', 'wind_speed', 'photosynthetic', 'sun_exposure_time', 'COz', 'soil_ph', 'weather_id', 'label']
{'picture_id': (34.0, 23524.0), 'temperature': (4.9, 44.6), 'humidity': (13.2, 98.9), 'illuminance': (0.0, 28165.0), 'soil_temperature': (0.0, 20.6), 'soil_humidity': (0.0, 58.5), 'pressure': (0.0, 3914.6), 'wind_speed': (0.0, 8.66), 'photosynthetic': (0.0, 1822.0), 'sun_exposure_time': (0.0, 9.9), 'COz': (0.0, 55000.0), 'soil_ph': (5.02, 14.0), 'weather_id': (45288.0, 65382.0), 'label': (0, 3)}


In [11]:
xiangguan_sensor = ['temperature', 'humidity', 'illuminance', 'soil_temperature', 'soil_humidity', 'pressure', 'wind_speed', 'photosynthetic', 'sun_exposure_time', 'COz', 'soil_ph']

df_xiangguan['sensor'] = df_xiangguan[xiangguan_sensor].values.tolist()
print("input dim:",len(xiangguan_sensor))

input dim: 11


In [13]:
df=df_xiangguan
if config.debug:
    df = df[:100]
df.shape

(17169, 21)

create folds

In [14]:
skf = StratifiedKFold(n_splits=config.n_fold, shuffle=True, random_state=config.seed)  
for fold, (train_idx, val_idx) in enumerate(skf.split(df,df.label_name)):
    df.loc[val_idx, 'fold'] = fold
df.groupby(['fold'])['label_name'].count()# ???

fold
0.0    3434
1.0    3434
2.0    3434
3.0    3434
4.0    3433
Name: label_name, dtype: int64

In [15]:
df.head()
df.to_csv("test_fold.csv",index=False)

# dataset


In [16]:
myTransforms = transforms.Compose([
    transforms.Resize((config.img_size,config.img_size)),
    transforms.ToTensor(),
    transforms.Normalize(
    mean=[0.7136, 0.7118, 0.6788],
    std=[0.3338, 0.3453, 0.3020],
    
)
])

def load_img(path):
    img =  Image.open(path).convert('RGB')
    img = myTransforms(img)
    return img

class BuildDataset(torch.utils.data.Dataset):
    def __init__(self, df, label=True, transforms=None):
        self.df         = df
        self.label      = label
        self.sensors = df['sensor'].tolist()
        self.img_paths  = df['image_path'].tolist()   
        if self.label:
            self.labels = df['label'].tolist()
        self.transforms = transforms
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        img_path  = self.img_paths[index]
        img = load_img(img_path)
        sensor = self.sensors[index]
        sensor = torch.tensor(sensor).unsqueeze(0) #[1,n]
        if self.label:
            label = self.labels[index]
            return torch.tensor(img).to(torch.float), torch.tensor(sensor).to(torch.float),torch.tensor(label).to(torch.float)
        else:
            return torch.tensor(img).to(torch.float), torch.tensor(sensor).to(torch.float)

# dataloader

In [17]:
def fetch_dataloader(fold:int):
    train_df = df.query("fold!=@fold").reset_index(drop=True)

    valid_df = df.query("fold==@fold").reset_index(drop=True)
    print("train_df.shape:",train_df.shape)
    print("valid_df.shape:",valid_df.shape)

    train_data  = BuildDataset(df=train_df,label=True)
    valid_data = BuildDataset(df=valid_df,label=True)

    train_loader = DataLoader(train_data, batch_size=config.train_batch_size,shuffle=True)
    valid_loader = DataLoader(valid_data, batch_size=config.valid_batch_size,shuffle=False)
    # test_loader = DataLoader(test_data, batch_size=config.test_batch_size,shuffle=False)
    return train_loader,valid_loader


In [18]:
# train_dataset = BuildDataset(df=df)
# train_loader = DataLoader(train_dataset, batch_size=config.train_batch_size,shuffle=True)
# valid_loader = DataLoader(train_dataset, batch_size=config.valid_batch_size,shuffle=True)
train_loader,valid_loader = fetch_dataloader(fold=0)


train_df.shape: (13735, 22)
valid_df.shape: (3434, 22)


In [19]:
img,sensor,label = next(iter(train_loader))
print(img.shape)
print(sensor.shape)
print(label.shape)

/tmp/ipykernel_446673/355586058.py:36: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(img).to(torch.float), torch.tensor(sensor).to(torch.float),torch.tensor(label).to(torch.float)


torch.Size([32, 3, 384, 384])
torch.Size([32, 1, 11])
torch.Size([32])


# model

In [20]:

class sensorViLTransformerSS(nn.Module):

    def __init__(self,sensor_class_n,output_class_n):
        super().__init__()
        self.sensor_linear = nn.Linear(sensor_class_n,config.hidden_size) 

        self.token_type_embeddings = nn.Embedding(2, config.hidden_size)
        self.token_type_embeddings.apply(objectives.init_weights)

        self.transformer = getattr(vit, config.vit)(
                pretrained=True, config=vars(config)
            )
       
        self.dense = nn.Linear(config.hidden_size, config.hidden_size)
        self.activation = nn.Tanh()


        self.pooler = heads.Pooler(config.hidden_size)

        # self.pooler.apply(objectives.init_weights)
        self.classifier = nn.Linear(config.hidden_size,output_class_n)

        hs = config.hidden_size


    def infer(
        self,
        batch,
        mask_image=False,
        image_token_type_idx=1,
        image_embeds=None,
        image_masks=None,
    ):
        sensor = batch['sensor'].to(config.device)
        sensor_embeds = self.sensor_linear(sensor) # input[1,1,12]  output[1,1,768]
        

        if image_embeds is None and image_masks is None:
            img = batch["image"].to(config.device)
       
            (
                image_embeds, # torch.Size([1, 217, 768])
                image_masks, # torch.Size([1, 217])
                patch_index,
                image_labels,
            ) = self.transformer.visual_embed(
                img,
                max_image_len=config.max_image_len,
                mask_it=mask_image,
            )
        else:
            patch_index, image_labels = (
                None,
                None,
            )
        # 用embedding对数据输入预处理，降低维度
        image_embeds = image_embeds + self.token_type_embeddings(
                torch.full_like(image_masks, image_token_type_idx)
            )
        # sensor_masks = batch['sensor_masks'] # 序列数量
        batch_size = img.shape[0]
        sensor_masks = torch.ones(batch_size,1).to(config.device) # 序列数量
        image_masks = image_masks.to(config.device)
        co_embeds = torch.cat([sensor_embeds, image_embeds], dim=1) # torch.Size([1, 240, 768]) ->240=217+23
        co_masks = torch.cat([sensor_masks, image_masks], dim=1) # torch.Size([1, 240])

        x = co_embeds.to(config.device)

        for i, blk in enumerate(self.transformer.blocks):
            blk = blk.to(config.device)
            x, _attn = blk(x, mask=co_masks)

        x = self.transformer.norm(x) # torch.Size([1, 240, 768])
        sensor_feats, image_feats = ( # torch.Size([1, 23, 768]),torch.Size([1, 217, 768])
            x[:, : sensor_embeds.shape[1]], # 后面字数输出23维
            x[:, sensor_embeds.shape[1] :], # 前面图片输出217维
        )
        cls_feats = self.pooler(x) # torch.Size([1, 768])
        # cls_feats = self.dense(x)
        # cls_feats = self.activation(cls_feats)
        cls_output = self.classifier(cls_feats)
        # m = nn.Softmax(dim=1)
        
        m = nn.Sigmoid()
        cls_output = m(cls_output)
        
        ret = {
           "sensor_feats":sensor_feats,
            "image_feats": image_feats,
            "cls_feats": cls_feats, # class features
            "raw_cls_feats": x[:, 0],
            "image_labels": image_labels,
            "image_masks": image_masks,
           
            "patch_index": patch_index,

            "cls_output":cls_output,
        }

        return ret

    def forward(self, batch):
        ret = dict()
        
        ret.update(self.infer(batch))
        return ret


## model build

In [21]:
model = sensorViLTransformerSS(sensor_class_n= config.senser_input_num,output_class_n = 1)
model.to(config.device)
print(config.device)
for i,m in enumerate(model.modules()):
    print(i,m)

No pretrained weights exist or were found for this model. Using random initialization.


cuda:0
0 sensorViLTransformerSS(
  (sensor_linear): Linear(in_features=11, out_features=768, bias=True)
  (token_type_embeddings): Embedding(2, 768)
  (transformer): VisionTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 768, kernel_size=(32, 32), stride=(32, 32))
    )
    (pos_drop): Dropout(p=0.1, inplace=False)
    (blocks): ModuleList(
      (0): Block(
        (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=768, out_features=2304, bias=True)
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=768, out_features=768, bias=True)
          (proj_drop): Dropout(p=0.1, inplace=False)
        )
        (drop_path): Identity()
        (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): Mlp(
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (act): GELU(approximate=none)
          (fc2): Linear(in_fea

# 损失函数

In [22]:
criterion = F.mse_loss #均方误差损失函数
# criterion = F.mae_loss

# train one epoch

In [23]:



def train_one_epoch(model, optimizer, scheduler, dataloader, device, epoch):
    model.train()
    dataset_size = 0
    running_loss = 0.0
    
    pbar = tqdm(enumerate(dataloader), total=len(dataloader), desc='Train ')
    for step, (img, sensor,label) in pbar:         
        # img = img.to(device, dtype=torch.float)
        # sensor  = sensor.to(device, dtype=torch.float)
        # label  = label.to(device, dtype=torch.float)
        batch_size = img.size(0)
        
        batch = {"image":img,"sensor":sensor}

        y_pred = model(batch)
        label = label.to(config.device).unsqueeze(1)
        loss = criterion(y_pred['cls_output'], label)
        
        #一坨优化
        optimizer.zero_grad()#每一次反向传播之前都要归零梯度
        loss.backward()      #反向传播
        optimizer.step()     #固定写法
        scheduler.step()
     
        running_loss += (loss.item() * batch_size)
        dataset_size += batch_size
        epoch_loss = running_loss / dataset_size
        mem = torch.cuda.memory_reserved() / 1E9 if torch.cuda.is_available() else 0
        current_lr = optimizer.param_groups[0]['lr']
        pbar.set_postfix(train_loss=f'{epoch_loss:0.4f}',
                        lr=f'{current_lr:0.5f}',
                        gpu_mem=f'{mem:0.2f} GB')

    
        
        
    torch.cuda.empty_cache()
    gc.collect()
    
    return epoch_loss

# valid one epoch

In [24]:
@torch.no_grad()
def valid_one_epoch(model, dataloader, device, optimizer):
    model.eval()
    
    dataset_size = 0
    running_loss = 0.0
    
    val_scores = []
    
    pbar = tqdm(enumerate(dataloader), total=len(dataloader), desc='Valid ')
    for step, (img, sensor,label) in pbar:               
        
        
        batch_size = img.size(0)
        batch = {"image":img,"sensor":sensor}

        y_pred  = model(batch)
        label = label.to(config.device).unsqueeze(1)

        loss = criterion(y_pred['cls_output'], label)
        
        running_loss += (loss.item() * batch_size)
        dataset_size += batch_size
        
        epoch_loss = running_loss / dataset_size
        
        
        mem = torch.cuda.memory_reserved() / 1E9 if torch.cuda.is_available() else 0
        current_lr = optimizer.param_groups[0]['lr']
        pbar.set_postfix(valid_loss=f'{epoch_loss:0.4f}',
                        lr=f'{current_lr:0.5f}',
                        gpu_memory=f'{mem:0.2f} GB')
    torch.cuda.empty_cache()
    gc.collect()
    
    return epoch_loss

# train

In [25]:

def run_training(model, optimizer, scheduler, device, num_epochs):
     # init wandb
    run = wandb.init(project="vilt",
                    config={k: v for k, v in dict(vars(config)).items() if '__' not in k},
                    # config={k: v for k, v in dict(config).items() if '__' not in k},
                    anonymous=anonymous,
                    # name=f"vilt|fold-{config.valid_fold}",
                    name=f"vilt|",
                    # group=config.wandb_group,
                    )
    wandb.watch(model, log_freq=100)

    best_loss = 9999
    best_valid_loss = 9999
    history = defaultdict(list)
    if torch.cuda.is_available():
        print("cuda: {}\n".format(torch.cuda.get_device_name()))
    
    for epoch in range(1, num_epochs + 1): 
        gc.collect()
        print(f'Epoch {epoch}/{num_epochs}', end='')
        train_loss = train_one_epoch(model, optimizer, scheduler, 
                                           dataloader=train_loader, 
                                           device=device, epoch=epoch)
        val_loss = valid_one_epoch(model,valid_loader,device=device,optimizer=optimizer)
        history['Train Loss'].append(train_loss)
        history['Valid Loss'].append(val_loss)

        wandb.log({"Train Loss": train_loss,
                    "Valid Loss": val_loss,
                "lr": scheduler.get_last_lr()[0]
                })
        if best_valid_loss > val_loss:
            best_valid_loss = val_loss
            # model_file_path = os.path.join(wandb.run.dir,"epoch-{}-{}.bin".format(epoch,wandb.run.id))
            model_file_path = os.path.join(wandb.run.dir,"epoch-best.bin")
            run.summary["Best Epoch"] = epoch
            torch.save(model.state_dict(), model_file_path)
            print("model save to", model_file_path)
    run.finish()
    return model, history

optimizer

In [26]:
optimizer = optim.Adam(model.parameters(), lr=config.learning_rate, weight_decay=config.weight_decay)
scheduler = lr_scheduler.CosineAnnealingLR(optimizer,T_max=config.T_max, 
                                                   eta_min=1e-5)


run train

In [27]:

# model, history = run_training(model, optimizer, scheduler,device=config.device,num_epochs=config.max_epoch)



# infer

In [28]:
for (img,sensor,label) in valid_loader:
    print(img.shape,sensor.shape,label)
    break

torch.Size([4, 3, 384, 384]) torch.Size([4, 1, 11]) tensor([0., 0., 0., 0.])


/tmp/ipykernel_446673/355586058.py:36: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(img).to(torch.float), torch.tensor(sensor).to(torch.float),torch.tensor(label).to(torch.float)


In [29]:
# torch.save(model.state_dict(), 'embedding_test_dict.pt')
# print(model)

model.load_state_dict(torch.load("/home/junsheng/ViLT/wandb/offline-run-20220811_120519-nzfb1xoz/files/epoch-best.bin"))
model.eval()
device = config.device
model.to(device)
def infer(img_filename, sensor):
    try:
        img_path = os.path.join('pictures',img_filename)
        image = Image.open(img_path).convert("RGB")
        img = pixelbert_transform(size=384)(image) # 将图像数据归一化torch.Size([3, 384, 576])
        img = torch.tensor(img)
        img = torch.unsqueeze(img, 0) # torch.Size([1, 3, 384, 576])
        img = img.to(device)
        print("img.shape:",img.shape)
    except :
        print("图片加载失败！")
        raise

    batch = dict()
    batch["image"] = img

    batch['sensor_masks'] = torch.ones(1,1).to(device)
    with torch.no_grad():
        batch['sensor'] = sensor.to(device)       
        infer = model(batch)

        print(infer)
        sensor_emb, img_emb = infer["sensor_feats"], infer["image_feats"]# torch.Size([1, 23, 768]) torch.Size([1, 217, 768])
        cls_output = infer['cls_output']
        

    return [cls_output]


random test

In [30]:

examples=[
            "/home/junsheng/data/xiangguan/pic/xiangguanD4-2021-05-24-10-00-25.jpeg", #0
            
            "/home/junsheng/data/xiangguan/pic/xiangguanD4-2021-07-18-04-22-30-preset-18.jpeg", # 3
    ]



n = 1
sensor = torch.rand(config.senser_input_num)
# sensor = torch.ones(config.senser_input_num)
print(sensor)
sensor =  torch.tensor(sensor).unsqueeze(0).unsqueeze(0) # torch.Size([1, 1, 3])
out = infer(examples[0],sensor)
# print("out:",out,"000\n")
# print("out0.shape:",out[0].shape)
# cv2.imwrite('output.png',out[0])



tensor([0.9552, 0.2386, 0.8053, 0.8996, 0.8717, 0.2519, 0.1701, 0.5269, 0.7088,
        0.9925, 0.1388])
img.shape: torch.Size([1, 3, 352, 608])


/tmp/ipykernel_446673/3499233738.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sensor =  torch.tensor(sensor).unsqueeze(0).unsqueeze(0) # torch.Size([1, 1, 3])


{'sensor_feats': tensor([[[ 3.0723e-06, -1.7554e-07,  1.5229e-07, -2.1967e-08, -2.2484e-06,
           3.2141e-06,  6.0833e-06, -5.4010e-09, -1.9050e-07, -7.8322e-07,
          -1.5750e-07,  3.6265e-06, -3.8516e-08, -1.0081e-05, -1.4026e-08,
          -1.8135e-07, -2.7653e-06,  9.1704e-09, -2.7459e-06, -5.0790e-09,
           9.4938e-07,  4.1226e-09,  7.3300e-08, -1.0659e-08,  6.3456e-07,
           3.1712e-06, -2.6293e-06, -1.0807e-09,  2.9015e-06,  4.4993e-06,
          -9.2933e-06,  1.0645e-06, -4.3114e-07,  2.2734e-08, -9.7144e-07,
           1.0887e-06,  6.2589e-08, -7.6105e-08, -2.9577e-09,  2.1593e-06,
           7.3475e-08,  8.4675e-06,  3.4534e-08, -2.6103e-07, -4.6536e-06,
           1.3890e-08,  3.9890e-08, -4.9402e-06, -2.0919e-06, -2.7671e-06,
           1.0942e-07, -4.3214e-06, -3.1810e-08, -1.3133e-09, -2.3513e-06,
          -1.7040e-09, -9.4722e-06,  2.6569e-06,  6.1225e-07, -5.3678e-08,
           1.0648e-05, -2.6208e-07, -3.8011e-07,  1.9068e-08, -7.3565e-08,
        

/home/junsheng/.conda/envs/pytorch_junsheng_39/lib/python3.9/site-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [31]:
out

[tensor([[0.3515]], device='cuda:0')]

In [32]:
print(out[0].cpu().numpy()[0][0])
#0.00031266143

0.35151166


test by valid

选择三组生长期不同的数据去验证训练的结果

In [43]:
df_test = df.query("fold==0").reset_index(drop=True)
df_test.to_csv("test_by_valid.csv",index=False)
sensor_test_list = df_test.sensor.tolist()
image_test_list = df_test.image_path.tolist()

In [45]:
idx = 64
sensor =  torch.tensor(sensor_test_list[idx]).unsqueeze(0).unsqueeze(0)
out = infer(image_test_list[idx],sensor)

img.shape: torch.Size([1, 3, 352, 608])
{'sensor_feats': tensor([[[ 3.0731e-06, -1.7551e-07,  1.5700e-07, -2.3826e-08, -2.2488e-06,
           3.2136e-06,  6.0759e-06, -5.3882e-09, -1.9196e-07, -7.7300e-07,
          -1.5751e-07,  3.6218e-06, -3.8495e-08, -1.0082e-05, -1.3897e-08,
          -1.8135e-07, -2.7734e-06,  9.1711e-09, -2.7478e-06, -6.8396e-09,
           9.5117e-07,  3.9358e-09,  7.3520e-08, -1.0666e-08,  6.3484e-07,
           3.1724e-06, -2.6400e-06, -1.0313e-09,  2.8941e-06,  4.4986e-06,
          -9.2994e-06,  1.0640e-06, -4.3119e-07,  2.4046e-08, -9.7306e-07,
           1.0860e-06,  7.6602e-08, -7.6102e-08, -2.9333e-09,  2.1590e-06,
           7.3389e-08,  8.4670e-06,  3.4530e-08, -2.6061e-07, -4.6415e-06,
           1.3894e-08, -6.2063e-08, -1.9665e-06, -2.1450e-06, -2.7528e-06,
           1.0902e-07, -4.3158e-06, -3.1787e-08, -1.1309e-09, -2.3554e-06,
          -1.7433e-09, -9.4328e-06,  2.6709e-06,  3.7688e-07, -5.3718e-08,
           1.0649e-05, -2.6325e-07, -3.8030

/tmp/ipykernel_446673/164085027.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  img = torch.tensor(img)


In [46]:
idx = 876
sensor =  torch.tensor(sensor_test_list[idx]).unsqueeze(0).unsqueeze(0)
out = infer(image_test_list[idx],sensor)

img.shape: torch.Size([1, 3, 352, 608])
{'sensor_feats': tensor([[[ 3.0827e-06, -1.7530e-07,  1.5452e-07, -2.2674e-08, -2.2496e-06,
           3.2150e-06,  6.0465e-06, -5.3801e-09, -1.8847e-07, -7.7690e-07,
          -1.5788e-07,  3.6213e-06, -3.8558e-08, -1.0063e-05, -1.4680e-08,
          -1.8126e-07, -2.7730e-06,  9.1713e-09, -2.7553e-06, -5.3852e-09,
           9.5103e-07,  3.8168e-09,  7.2278e-08, -1.0654e-08,  6.3138e-07,
           3.1689e-06, -2.6612e-06, -1.0689e-09,  2.8926e-06,  4.4980e-06,
          -9.2651e-06,  1.0652e-06, -4.1821e-07,  2.2096e-08, -9.7153e-07,
           1.0851e-06,  7.0076e-08, -7.6146e-08, -2.9167e-09,  2.1692e-06,
           7.3493e-08,  8.4667e-06,  3.4533e-08, -2.6099e-07, -4.6151e-06,
           1.3888e-08,  3.3025e-07, -5.3568e-06, -2.0778e-06, -2.7470e-06,
           1.0874e-07, -4.3026e-06, -3.1775e-08, -1.3380e-09, -2.3559e-06,
          -1.7084e-09, -9.4255e-06,  2.6611e-06,  7.8697e-07, -5.3748e-08,
           1.0649e-05, -2.6270e-07, -3.8041

/tmp/ipykernel_446673/164085027.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  img = torch.tensor(img)


In [47]:
idx = 1817
sensor =  torch.tensor(sensor_test_list[idx]).unsqueeze(0).unsqueeze(0)
out = infer(image_test_list[idx],sensor)

img.shape: torch.Size([1, 3, 352, 608])
{'sensor_feats': tensor([[[ 3.0705e-06, -1.7521e-07,  1.5456e-07, -2.1202e-08, -2.2476e-06,
           3.2173e-06,  6.0811e-06, -5.3703e-09, -1.9807e-07, -7.9415e-07,
          -1.5692e-07,  3.6335e-06, -3.8573e-08, -1.0093e-05, -1.3580e-08,
          -1.8139e-07, -2.7724e-06,  9.1703e-09, -2.7452e-06, -5.7367e-09,
           9.5353e-07,  3.9170e-09,  8.3201e-08, -1.0664e-08,  6.3288e-07,
           3.1788e-06, -2.6123e-06, -1.0732e-09,  2.9125e-06,  4.4989e-06,
          -9.3056e-06,  1.0643e-06, -4.5918e-07,  2.2646e-08, -9.7590e-07,
           1.0921e-06,  5.4508e-08, -7.6128e-08, -2.9861e-09,  2.1609e-06,
           7.3610e-08,  8.4742e-06,  3.4496e-08, -2.6085e-07, -4.6524e-06,
           1.3929e-08, -5.0008e-07, -2.3619e-06, -2.0132e-06, -2.7828e-06,
           1.0898e-07, -4.3159e-06, -3.1896e-08, -1.0233e-09, -2.3536e-06,
          -1.7547e-09, -9.5498e-06,  2.6606e-06,  2.5922e-07, -5.3656e-08,
           1.0648e-05, -2.6302e-07, -3.8063

/tmp/ipykernel_446673/164085027.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  img = torch.tensor(img)
